In [1]:
import numpy as np
import matplotlib.pyplot as plt

x = np.array(([3,5],[5,1],[10,2]),dtype=float)
y = np.array(([75],[82],[93]),dtype=float)
x = x/np.amax(x,axis=0)
y = y/100

class Neural_Network(object):
    def __init__(self):
        self.inputLayerSize = 2
        self.outputLayerSize = 1
        self.hiddenLayerSize = 3
        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
    def forward(self,x):
        self.z2 = np.dot(x,self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2,self.W2)
        yHat = self.sigmoid(self.z3)
        return yHat
    
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def costFunction(self, x, y):
        self.yHat = self.forward(x)
        J = 0.5*sum((y-self.yHat)**2)
        return J
    
    def costFunctionPrime(self, x,y):
        self.yHat = self.forward(x)
        delta3 = np.multiply(-(y-self.yHat),self.sigmoidPrime(self.z3))
        dJdW2 = np.dot(self.a2.T,delta3)
        delta2 = np.dot(delta3,self.W2.T)*self.sigmoidPrime(self.z2)
        dJdW1 = np.dot(x.T,delta2)
        return dJdW1, dJdW2
    
    def getParams(self):
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params
    
    def setParams(self, params):
        W1_start = 0
        W1_end = self.hiddenLayerSize * self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayerSize , self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayerSize, self.outputLayerSize))
        
    def computeGradients(self, x, y):
        dJdW1, dJdW2 = self.costFunctionPrime(x, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

In [2]:
def computeNumericalGradient(N, x, y):
        paramsInitial = N.getParams()
        numgrad = np.zeros(paramsInitial.shape)
        perturb = np.zeros(paramsInitial.shape)
        e = 1e-4
        for p in range(len(paramsInitial)):
            perturb[p] = e
            N.setParams(paramsInitial + perturb)
            loss2 = N.costFunction(x, y)
            N.setParams(paramsInitial - perturb)
            loss1 = N.costFunction(x, y)
            numgrad[p] = (loss2 - loss1) / (2*e)
            perturb[p] = 0
        N.setParams(paramsInitial)

        return numgrad

In [3]:
NN = Neural_Network()

In [4]:
numgrad = computeNumericalGradient(NN, x, y)
numgrad

array([-0.03111554, -0.04116023,  0.02500021, -0.02072572, -0.02685856,
        0.01712275, -0.04528982, -0.06777929, -0.09659213])

In [5]:
grad = NN.computeGradients(x,y)
grad

array([-0.03111554, -0.04116023,  0.02500021, -0.02072572, -0.02685856,
        0.01712275, -0.04528982, -0.06777929, -0.09659213])

In [6]:
from scipy import optimize

In [7]:
class trainer(object):
    def __init__(self, N):
        self.N = N
        
    def callbackF(self, params):
        self.N.setParams(params)
        self.J.append(self.N.costFunction(self.x, self.y))   
        
    def costFunctionWrapper(self, params, x, y):
        self.N.setParams(params)
        cost = self.N.costFunction(x, y)
        grad = self.N.computeGradients(x,y)
        
        return cost, grad
        
    def train(self, x, y):
        self.x = x
        self.y = y
        self.J = []
        params0 = self.N.getParams()
        options = {'maxiter': 200, 'disp' : True}
        _res = optimize.minimize(self.costFunctionWrapper, params0, jac=True, method='BFGS', \
                                 args=(x, y), options=options, callback=self.callbackF)
        self.N.setParams(_res.x)
        self.optimizationResults = _res

In [8]:
NN = Neural_Network()

In [9]:
T = trainer(NN)

In [10]:
T.train(x,y)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 81
         Function evaluations: 91
         Gradient evaluations: 91


In [12]:
plt.plot(T.J)
plt.grid(1)
plt.xlabel('Iterations')
plt.ylabel('Cost')

In [13]:
NN.costFunctionPrime(x,y)

(array([[  5.29983176e-06,   3.17676185e-06,  -6.31911986e-06],
        [  4.11576408e-06,   2.24325594e-06,  -4.09299827e-06]]),
 array([[  6.64991445e-06],
        [  6.95335645e-06],
        [  3.14118945e-06]]))

In [14]:
NN.forward(x)

array([[ 0.75001609],
       [ 0.82004072],
       [ 0.93004977]])

In [15]:
y

array([[ 0.75],
       [ 0.82],
       [ 0.93]])

In [18]:
hoursSleep = np.linspace(0, 10, 100)
hoursStudy = np.linspace(0, 5, 100)
hoursSleepNorm = hoursSleep/10.
hoursStudyNorm = hoursStudy/5.
a, b  = np.meshgrid(hoursSleepNorm, hoursStudyNorm)
allInputs = np.zeros((a.size, 2))
allInputs[:, 0] = a.ravel()
allInputs[:, 1] = b.ravel()

In [19]:
allOutputs = NN.forward(allInputs)

In [20]:
yy = np.dot(hoursStudy.reshape(100,1), np.ones((1,100)))
xx = np.dot(hoursSleep.reshape(100,1), np.ones((1,100))).T

CS = plt.contour(xx,yy,100*allOutputs.reshape(100, 100))
plt.clabel(CS, inline=1, fontsize=10)
plt.xlabel('Hours Sleep')
plt.ylabel('Hours Study')

In [22]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.gca(projection='3d')

surf = ax.plot_surface(xx, yy, 100*allOutputs.reshape(100, 100), \
                       cmap=cm.jet)

ax.set_xlabel('Hours Sleep')
ax.set_ylabel('Hours Study')
ax.set_zlabel('Test Score')

ImportError: No module named PyQt4